# Auto type class benchmark

This benchmark aims at comparing various changes in the [`Lazy` implicit helper](https://github.com/milessabin/shapeless/blob/shapeless-2.2.5/core/src/main/scala/shapeless/lazy.scala) from [shapeless](https://github.com/milessabin/shapeless), done in the [lazyextensions](https://github.com/milessabin/shapeless/tree/topic/lazyextensions) branch, and various ways of using these from a [(necessarily) modified version](https://github.com/alexarchambault/upickle-pprint) of [upickle](https://github.com/lihaoyi/upickle-pprint). Advantage of using upickle for this are:
* it has a comprehensive [test suite](https://github.com/lihaoyi/upickle-pprint/tree/0.3.4/upickle/shared/src/test/scala), and
* it derives its type classes using [macros](https://github.com/lihaoyi/upickle-pprint/blob/0.3.4/upickle/shared/src/main/scala/upickle/Api.scala#L75-L80), which the type level / shapeless-only approach can be benchmarked against.

## Running the benchmark

Clone the `auto-type-class-benchmark` repository with

    git clone https://github.com/alexarchambault/auto-type-class-benchmark.git
    
From its directory, run the `shapeless-publish-local` script, that publishes locally various versions of the [lazyextensions](https://github.com/milessabin/shapeless/tree/topic/lazyextensions) branch, suffixing their versions with their commits (no risk of collision with standard versions),

    cd auto-type-class-benchmark
    util/shapeless-publish-local

Then, run the `run-benchmark` script, that clones the [upickle modified version](https://github.com/alexarchambault/upickle-pprint) repository, and runs various versions of it against the various versions of the lazyextensions branch from above,

    util/run-benchmark
    
The above command can take some time. As a reference, you can run

    git clone https://github.com/alexarchambault/upickle-pprint.git -b master upickle-pprint-master
    cd upickle-pprint-master
    sbt upickleJVM/test

If this SBT command takes about 30 seconds, the whole benchmark suite should take approx. 5 hours to run.

The `run-benchmark` script runs 5 times `sbt upickleJVM/test` for each upickle and shapeless versions, and saves their output in the `results` directory. These outputs can then be parsed by the `parse-results` script to get a CSV of the various compilation times,

    util/parse-results results > results.csv

like [the one](https://github.com/alexarchambault/auto-type-class-benchmark/blob/master/results-1.csv) from the auto-type-class-benchmark repository. This CSV can then be parsed... from this very notebook. It requires [IPython](https://github.com/ipython/ipython) and [jupyter-scala](https://github.com/alexarchambault/jupyter-scala) to be installed. Edit / inspect it by running

    ipython notebook
  
from the `auto-type-class-benchmark` directory, and opening `Analysis.ipynb` from the newly opened browser tab.

## Analysis

### Preamble

In [1]:
load.ivy(
  "com.twitter" %% "algebird-core" % "0.11.0",
  "com.github.alexarchambault" %% "uplot-highcharts" % "0.1.0-SNAPSHOT"
)

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Thu Jun 04 00:37:50 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#uplot-highcharts_2.11;0.1.0-SNAPSHOT from sonatype-snapshots, using Wed Feb 11 20:25:55 CET 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#uplot-

In [2]:
import scala.io.Source
import com.twitter.algebird.{ Moments, Monoid }
import uplot._

import scala.io.Source
import com.twitter.algebird.{ Moments, Monoid }
import uplot._

### Get shapeless / upickle commits and their description

In [3]:
private val commits = Source.fromFile("util/commits").getLines().toArray.toSeq

val (Seq(_, shapelessCommits @ _*), Seq(_, _, upickleCommits @ _*)) = commits
    .map(_.split(" ", 2).toSeq)
    .collect{
      case Seq(c, d) => c -> d
      case Seq(other) => other -> ""
    }
    .splitAt(commits.indexWhere(_.isEmpty))
val shapelessCommitsMap = shapelessCommits.toMap
val upickleCommitsMap = upickleCommits.toMap

shapelessCommits: Seq[(String, String)] = ArrayBuffer(
  ("3fd06cd0", "0. master"),
  ("0fa6a9d4", "1. v0"),
  ("2085addf", "2. Strict Implicit"),
  ("22b85cb8", "3. inlining"),
  ("cd05b4af", "4. cached"),
  ("0760706f", "5. Don't clear scalac implicit cache"),
  ("41f6f6c5", "6. Revert inlining")
)
upickleCommits: Seq[(String, String)] = ArrayBuffer(
  ("19052c3f", "0. master"),
  ("2a63bef3", "1. Lazy only, Implicit, no Strict or Cached"),
  ("4266949f", "2. No Lazy tails"),
  ("aabcc2a4", "3. Helper type classes for tails"),
  ("aa82a2ba", "4. Usual implicit prioritisation"),
  ("b823fab0", "5. No superfluous Lazy wrapper"),
  ("e5954868", "6. Using Strict"),
  ("5938da82", "7. Cached"),
  ("b4d36c6e", "8. All Strict replaced by Lazy")
)
shapelessCommitsMap: Map[String, String] = Map(
  "22b85cb8" -> "3. inlining",
  "0fa6a9d4" -> "1. v0",
  "3fd06cd0" -> "0. master",
  "0760706f" -> "5. Don't clear scalac implicit cache",
  "2085addf" -> "2. Strict Implicit",
  "41f6f6c5" -> "6. R

### Helper class

In [4]:
case class SingleRun(shapelessCommit: String, upickleCommit: String, run: Int, timeSec: Int) {
  def key = (shapelessCommit, upickleCommit)
}

object SingleRun {
  def fromLine(line: Seq[String]) = {
    assert(line.length == 4)
    SingleRun(line(0), line(1), line(2).toInt, line(3).toInt)
  }
}

defined class SingleRun
defined object SingleRun

### Compute compilation time statistics

Reads the CSV generated when running the benchmarks (edit its path below if you ran them yourself).

Results are put in the `stats` map (keys: (shapeless commit, upickle commit), values: (mean compilation time in seconds, its std deviation)).

In [5]:
val csvPath = "results-1.csv"

val stats = Source.fromFile(csvPath)
  .getLines()
  .toList
  .map(_.split(';').toSeq)
  .drop(1) // drop header
  .map(SingleRun.fromLine)
  .groupBy(_.key)
  .mapValues{ l =>
    val m = Monoid.sum(l.map(run => Moments(run.timeSec)))
    (m.mean, m.stddev)
  }

csvPath: String = "results-1.csv"
stats: Map[(String, String), (Double, Double)] = Map(
  ("0760706f", "b4d36c6e") -> (96.8, 2.227105745132009),
  ("22b85cb8", "e5954868") -> (99.0, 2.756809750418044),
  ("2085addf", "b823fab0") -> (105.0, 1.8973665961010278),
  ("0fa6a9d4", "4266949f") -> (156.4, 2.65329983228432),
  ("cd05b4af", "b4d36c6e") -> (94.2, 1.1661903789690597),
  ("0fa6a9d4", "aa82a2ba") -> (117.8, 2.4819347291981715),
  ("0fa6a9d4", "aabcc2a4") -> (139.8, 3.187475490101844),
...

### Put the statistics in shape

In [6]:
val upickleStats = shapelessCommits
  .map(t => t -> 
     stats
       .collect{ case ((s, u), stats) if s == t._1 =>
         (u, upickleCommitsMap(u)) -> stats
       }
       .toList
       .sortBy(kv => upickleCommits.indexOf(kv._1))
  )
val shapelessStats = upickleCommits
  .map(t => t -> 
     stats
       .collect{ case ((s, u), stats) if u == t._1 =>
         (s, shapelessCommitsMap(s)) -> stats
       }
       .toList
       .sortBy(kv => shapelessCommits.indexOf(kv._1))
  )

upickleStats: Seq[((String, String), List[((String, String), (Double, Double))])] = ArrayBuffer(
  (
    ("3fd06cd0", "0. master"),
    List(
      (
        ("19052c3f", "0. master"),
        (32.6, 1.019803902718557)
      )
    )
  ),
  (
    ("0fa6a9d4", "1. v0"),
    List(
      (
        ("19052c3f", "0. master"),
...
shapelessStats: Seq[((String, String), List[((String, String), (Double, Double))])] = ArrayBuffer(
  (
    ("19052c3f", "0. master"),
    List(
      (
        ("3fd06cd0", "0. master"),
        (32.6, 1.019803902718557)
      ),
      (
        ("0fa6a9d4", "1. v0"),
        (32.8, 0.7483314773547879)
      ),
      (
        ("2085addf", "2. Strict Implicit"),
        (33.0, 1.0954451150103328)
...

### Helper / preamble for plotting

In [7]:
display.html(
  <div>
    <script src="http://code.highcharts.com/stock/highstock.js"></script>
    <script src="http://code.highcharts.com/stock/modules/exporting.js"></script>
    <script src="http://www.highcharts.com/js/themes/grid.js"></script>
  </div>
)

def plot(
  stats: Seq[((String, String), List[((String, String), (Double, Double))])],
  commits: Seq[(String, String)]
) = Plot(
  data = stats.map{case ((_, k), v) =>
    val map = v.toMap
    Data(
      x = (1 until commits.length).map(_.toDouble),
      y = commits.drop(1).map(c => map.get(c).fold(0.0)(_._1)),
      options = Seq(
        Data.Legend(k)
      )
    )
  },
  options = Seq(
    Plot.YLim(0, 180)
  )
)

defined function plot

## Results

Each line / color corresponds to a modified upickle version above, from master (0, blue) to a Lazy only one (8, dark yellow), including the main most optimized one (7, "Cached", dark gray). (Hover over the names in the legend at the bottom of the graph to highlight the various lines.)

See [this file](https://github.com/alexarchambault/auto-type-class-benchmark/blob/master/util/commits) for a full description of the commits.

The X axis corresponds to lazyextension branch versions, from a v0 (x=1), to the main one (x=4), then reverting some of its changes (x=5, don't clear scalac implit cache, and 6, don't inline results in Lazy macro output).

In [8]:
val id = "shapelessGraph"
display.html(<div id={id}></div>)
display.js(uplot.Highcharts.plotHighchartsJs(plot(shapelessStats, shapelessCommits), id))

id: String = "shapelessGraph"

Main points:
* going from x=4 to x=5 reverts clearing the implicit cache **of scalac** (see [here](https://github.com/milessabin/shapeless/blob/cd05b4af11be3bebccd632dbe4d4a63b8107b397/core/src/main/scala/shapeless/lazy.scala#L208-L210)). Reverting this slows things either sharply (1 - orange, 3 - red), or just a little. Clearing the scalac implicit cache only speeds up things in this benchmark, sometimes markedly, sometimes less.

* going from x=5 to x=6 reverts inlining of results in the output of the Lazy macro. This only slows things, sharply (1 - orange), moderately (2 - green, 3 - red, 4 - violet, 6 - pink), or just a little (the others). Inlining things only speeds up things in this benchmark, sometimes markedly too, sometimes less.

* the "drift" in the colored lines corresponds to speed ups of the various changes in the deriving type classes, in the modified version of upickle-pprint. By only changing things on its side, one can get a big chunk of the speed up gains (from 1. orange, to 6. pink). Compilation times are slightly better with the optimized version of the lazyextensions branch (x=4). They can be made a bit better again with caching / inlining (4. grey, requires x >= 4).

In [ ]:
// Alternative graph

// val id = "upickleGraph"
// display.html(<div id={id}></div>)
// display.js(uplot.Highcharts.plotHighchartsJs(plot(upickleStats, upickleCommits), id))